In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

from collections import defaultdict
from ukbb_recessive.regression.regressions import run_regressions, save_table_for_paper, get_formula, get_target_family

# Create regressions datasets

In [ ]:
output_path = ".../ukbb_recessive/data/tables/sampling/genes"
datasets_path = '.../450k/datasets/sampling/genes'

datasets_dict = {
    'Roulette': pd.read_csv(f"{datasets_path}/Roulette.csv", sep='\t'),
}

# samples of interest: European & non-related without hom and comp_het
european_non_rel_samples = (
    ".../450k/samples/european_non_related_no_withdrawal_to_include_450k.no_hom_comp_het.txt"
)

with open(european_non_rel_samples, 'r') as f:
    european_non_rel_samples = [l.strip() for l in f.readlines()]

print (f"Number of european non-related samples without hom and comp_het: {len(european_non_rel_samples)}\n")

for dataset_name, dataset in datasets_dict.items():
    print (f"Number of samples in {dataset_name}: {datasets_dict[dataset_name].shape[0]}")
    datasets_dict[dataset_name] = dataset[dataset['eid'].astype(str).isin(european_non_rel_samples)].copy()
    print (f"Number of samples in {dataset_name} after filtration: {datasets_dict[dataset_name].shape[0]}")

In [ ]:
panels = ['ID_sampled'] + [f'AR_without_ID_{i}_sampled' for i in range(20)]

panel_s_het_list = ['s_het_recessive_' + panel for panel in panels]

print (f"Total amount of panels: ", len(panels), flush=True)

# Define analyses

In [ ]:
# regressions for 21 sampled panels
id_vs_other_sampling_regressions = {
    's_hets': panel_s_het_list,
    'targets': ['childlessness', 'years_of_edu', 'diagnosis_total_ICD10_cnt_log', 'fluid_intelligence_score', 'is_blond'],
    'families': ['binomial', 'gaussian', 'gaussian', 'gaussian', 'binomial'],
    'genders': ['all'], 
    'n_tests_correction': 10,
    'filter_dataset': lambda dataset: dataset
}

analyses = {
    'id_vs_other_sampling': id_vs_other_sampling_regressions
}

# Generate regressions

In [ ]:
for analysis, analysis_cfg in analyses.items():

    all_results = defaultdict(list)

    print (f"Running {analysis} analysis\n")

    # read analysis config
    s_hets = analysis_cfg['s_hets']
    targets = analysis_cfg['targets']
    families = analysis_cfg['families']
    genders = analysis_cfg['genders']   
    filter_dataset_func = analysis_cfg['filter_dataset']
    n_tests_correction = analysis_cfg['n_tests_correction']
     
    
    # iterate over different s-het datasets
    for dataset_key in datasets_dict:
        print ("\tProcessing", dataset_key, flush=True)

        # iterate over different s-het burdens
        for s_het in s_hets:
            if not (isinstance(s_het, list) or isinstance(s_het, tuple)) :
                s_het = [s_het]

            analysis_tag = f'{analysis} regressions on {str(s_het)}'.replace("'", '')

            print (f"\t\tRun {analysis_tag}")

            dataset = filter_dataset_func(datasets_dict[dataset_key]).copy()
            print (f"\t\t\tDataset size after filtration: {dataset.shape}")

            regression_result = run_regressions(dataset=dataset, 
                                                targets=targets, 
                                                families=families, 
                                                analysis_tag=analysis_tag, 
                                                genders=genders, 
                                                s_het_list=s_het, 
                                                tab_offset='\t\t\t')
            
            all_results[dataset_key].append(regression_result)

            print()

    all_results = {dataset_key: pd.concat(all_results[dataset_key]) for dataset_key in all_results}

    save_table_for_paper(all_results, f"{output_path}/table_{analysis}_analysis.xlsx", 
                         keep_effects=[], n_tests_correction=n_tests_correction)

    print()

In [ ]:
!ls -lah .../ukbb_recessive/data/tables/sampling/genes